In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of the network
model = Net()



100%|██████████| 9912422/9912422 [00:00<00:00, 148133923.05it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 70964085.43it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 77366752.77it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 16948869.01it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0

Epoch [1/10], Step [100/938], Loss: 0.6275
Epoch [1/10], Step [200/938], Loss: 0.3122
Epoch [1/10], Step [300/938], Loss: 0.2366
Epoch [1/10], Step [400/938], Loss: 0.1993
Epoch [1/10], Step [500/938], Loss: 0.1763
Epoch [1/10], Step [600/938], Loss: 0.1636
Epoch [1/10], Step [700/938], Loss: 0.1484
Epoch [1/10], Step [800/938], Loss: 0.1335
Epoch [1/10], Step [900/938], Loss: 0.1390
Epoch [2/10], Step [100/938], Loss: 0.1060
Epoch [2/10], Step [200/938], Loss: 0.1081
Epoch [2/10], Step [300/938], Loss: 0.0964
Epoch [2/10], Step [400/938], Loss: 0.0910
Epoch [2/10], Step [500/938], Loss: 0.0865
Epoch [2/10], Step [600/938], Loss: 0.0990
Epoch [2/10], Step [700/938], Loss: 0.0956
Epoch [2/10], Step [800/938], Loss: 0.0863
Epoch [2/10], Step [900/938], Loss: 0.0802
Epoch [3/10], Step [100/938], Loss: 0.0726
Epoch [3/10], Step [200/938], Loss: 0.0648
Epoch [3/10], Step [300/938], Loss: 0.0599
Epoch [3/10], Step [400/938], Loss: 0.0726
Epoch [3/10], Step [500/938], Loss: 0.0702
Epoch [3/10

In [ ]:
class MyFocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average = True):
        '''
        Input: Raw logits from the neural net
        Target: Long shape (N, )
        '''
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(self.alpha, (float, int)):
            self.alpha = torch.tensor([alpha, 1 - alpha]) # Minority class get alpha coeff
        if isinstance(self.alpha, list):
            self.alpha = torch.tensor(alpha)
        self.size_average = size_average

    def forward(self, input_, target):

        target = target.view(-1, 1)
        logpt = F.log_softmax(input_, -1)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if self.alpha is not None:
            if self.alpha.type() != input_.type():
                self.alpha = self.alpha.type_as(input_)
            at = self.alpha.gather(0, target.view(-1).long())
            logpt *= at
        loss = -((1-pt)**self.gamma)*logpt
        if self.size_average:
            return loss.mean()
        return loss.sum()


In [ ]:
import torch.nn.functional as F

In [ ]:
model = Net()

criterion = MyFocalLoss(gamma = 1, alpha = [0.2 for i in range(10)])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0

Epoch [1/10], Step [100/938], Loss: 0.0912
Epoch [1/10], Step [200/938], Loss: 0.0400
Epoch [1/10], Step [300/938], Loss: 0.0308
Epoch [1/10], Step [400/938], Loss: 0.0272
Epoch [1/10], Step [500/938], Loss: 0.0242
Epoch [1/10], Step [600/938], Loss: 0.0209
Epoch [1/10], Step [700/938], Loss: 0.0189
Epoch [1/10], Step [800/938], Loss: 0.0178
Epoch [1/10], Step [900/938], Loss: 0.0184
Epoch [2/10], Step [100/938], Loss: 0.0124
Epoch [2/10], Step [200/938], Loss: 0.0134
Epoch [2/10], Step [300/938], Loss: 0.0120
Epoch [2/10], Step [400/938], Loss: 0.0145
Epoch [2/10], Step [500/938], Loss: 0.0132
Epoch [2/10], Step [600/938], Loss: 0.0122
Epoch [2/10], Step [700/938], Loss: 0.0125
Epoch [2/10], Step [800/938], Loss: 0.0108
Epoch [2/10], Step [900/938], Loss: 0.0113
Epoch [3/10], Step [100/938], Loss: 0.0075
Epoch [3/10], Step [200/938], Loss: 0.0082
Epoch [3/10], Step [300/938], Loss: 0.0087
Epoch [3/10], Step [400/938], Loss: 0.0080
Epoch [3/10], Step [500/938], Loss: 0.0081
Epoch [3/10

In [ ]:
# Evaluation
# Cross entropy loss: 97.91%
# Focal Loss: 97.71%
# Focal Loss with alpha: 97.96
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")

Accuracy on the test set: 97.96%
